# Question Classification Notebook Using Transformers approach
@ author: Raby3


## Import libraries

In [ ]:
!pip install gdown
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
import gdown
from contractions import fix

# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Data Loading and Parsing

In [ ]:
%%capture
#Loading Videos and aligned transcriptions (Annotaions)
url = 'https://drive.google.com/drive/folders/1w_x5uJz6fVlVciSijUD7u_4xMNWFNKS-?usp=drive_link'
gdown.download_folder(url, quiet=True)

In [ ]:
def load_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            # Split label and question
            label, question = line.strip().split(' ', 1)
            data.append({"question": question, "label": label})
    return pd.DataFrame(data)

train_set = load_data("/content/data/train_set5.txt")
test_set = load_data("/content/data/test_set.txt")

print(train_set.head())


                                            question        label
0  How did serfdom develop in and then leave Russ...  DESC:manner
1   What films featured the character Popeye Doyle ?  ENTY:cremat
2  How can I find a list of celebrities ' real na...  DESC:manner
3  What fowl grabs the spotlight after the Chines...  ENTY:animal
4                    What is the full form of .com ?     ABBR:exp


## Text Preprocessing

1. Lowercased text.
2. Removed special characters and numbers using regex.
3. Tokenized text into words using nltk.
4. Removed stopwords
5. Applied lemmatization for text normalization.

In [ ]:
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))
wh_words = {"who", "what", "where", "when", "why", "how", "is", "are", "does", "do", "did","was"}
custom_stop_words = stop_words - wh_words

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = fix(text)
    words = word_tokenize(text)
    words = [word for word in words if word not in custom_stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

train_set['processed_question'] = train_set['question'].apply(preprocess_text)
test_set['processed_question'] = test_set['question'].apply(preprocess_text)


##Keep WH words##

In [ ]:
train_set['processed_question'].head(20)

,processed_question
0,how did serfdom develop leave russia
1,what film featured character popeye doyle
2,how find list celebrity real name
3,what fowl grab spotlight chinese year monkey
4,what is full form com
5,what contemptible scoundrel stole cork lunch
6,what team did baseball st louis brown become
7,what is oldest profession
8,what are liver enzyme
9,name scarfaced bounty hunter old west


## Label Splitting:

Split labels (e.g., DESC:manner) into category (DESC) and specific_type (manner), making the structure of your labels more granular.

In [ ]:
train_set[['category', 'specific_type']] = train_set['label'].str.split(':', expand=True)
test_set[['category', 'specific_type']] = test_set['label'].str.split(':', expand=True)

print(train_set[['label', 'category', 'specific_type']].head())


         label category specific_type
0  DESC:manner     DESC        manner
1  ENTY:cremat     ENTY        cremat
2  DESC:manner     DESC        manner
3  ENTY:animal     ENTY        animal
4     ABBR:exp     ABBR           exp


## Encoding:

Used LabelEncoder to convert textual labels (category, specific_type, and combined labels)

In [ ]:
category_encoder = LabelEncoder()
specific_type_encoder = LabelEncoder()

train_set['category_encoded'] = category_encoder.fit_transform(train_set['category'])
train_set['specific_type_encoded'] = specific_type_encoder.fit_transform(train_set['specific_type'])

test_set['category_encoded'] = category_encoder.transform(test_set['category'])
test_set['specific_type_encoded'] = specific_type_encoder.transform(test_set['specific_type'])

print(train_set[['category', 'specific_type', 'category_encoded', 'specific_type_encoded']].head())


  category specific_type  category_encoded  specific_type_encoded
0     DESC        manner                 1                     23
1     ENTY        cremat                 2                      8
2     DESC        manner                 1                     23
3     ENTY        animal                 2                      1
4     ABBR           exp                 0                     16


In [ ]:
train_set['combined_label'] = train_set['category'] + "_" + train_set['specific_type']
test_set['combined_label'] = test_set['category'] + "_" + test_set['specific_type']

combined_label_encoder = LabelEncoder()
train_set['combined_label_encoded'] = combined_label_encoder.fit_transform(train_set['combined_label'])
test_set['combined_label_encoded'] = combined_label_encoder.transform(test_set['combined_label'])

print(train_set[['combined_label', 'combined_label_encoded']].head())


  combined_label  combined_label_encoded
0    DESC_manner                       4
1    ENTY_cremat                       9
2    DESC_manner                       4
3    ENTY_animal                       6
4       ABBR_exp                       1


In [ ]:
train_set.head()

,question,label,processed_question,category,specific_type,category_encoded,specific_type_encoded,combined_label,combined_label_encoded
0,How did serfdom develop in and then leave Russ...,DESC:manner,how did serfdom develop leave russia,DESC,manner,1,23,DESC_manner,4
1,What films featured the character Popeye Doyle ?,ENTY:cremat,what film featured character popeye doyle,ENTY,cremat,2,8,ENTY_cremat,9
2,How can I find a list of celebrities ' real na...,DESC:manner,how find list celebrity real name,DESC,manner,1,23,DESC_manner,4
3,What fowl grabs the spotlight after the Chines...,ENTY:animal,what fowl grab spotlight chinese year monkey,ENTY,animal,2,1,ENTY_animal,6
4,What is the full form of .com ?,ABBR:exp,what is full form com,ABBR,exp,0,16,ABBR_exp,1


In [ ]:
print(train_set['category'].value_counts())
print(train_set['specific_type'].value_counts())


category
ENTY    1250
HUM     1223
DESC    1162
NUM      896
LOC      835
ABBR      86
Name: count, dtype: int64
specific_type
ind          962
other        733
def          421
count        363
desc         321
manner       276
date         218
cremat       207
reason       191
gr           189
country      155
city         129
animal       112
food         103
dismed       103
termeq        93
period        75
money         71
exp           70
state         66
sport         62
event         56
product       42
substance     41
color         40
techmeth      38
dist          34
veh           27
perc          27
word          26
title         25
mount         21
body          16
abb           16
lang          16
plant         13
volsize       13
weight        11
symbol        11
instru        10
letter         9
code           9
speed          9
temp           8
ord            6
religion       4
currency       4
Name: count, dtype: int64


## Observations
1. **Categories**
- The ENTY (Entity), HUM (Human), and DESC (Description) categories dominate the dataset.
- ABBR (Abbreviation) is severely underrepresented with only 86 instances, making it a minority class.

2. **Specific Types**
The specific_type distribution is quite imbalanced:
- ind, other, and def are the most common types.
- Some specific types, like currency, religion, ord, and temp, have fewer than 10 instances.

## Challenges
1. **Class Imbalance:**
Both category and specific_type have significant imbalances.
Minority classes may lead to poor model performance for those classes.

2. **Granularity:**
Some specific types, such as techmeth and volsize, are too granular, which may increase the complexity of classification.

In [ ]:
print(f"Category classes: {category_encoder.classes_}")
print(f"Specific type classes: {specific_type_encoder.classes_}")
print(f"Combined label classes: {combined_label_encoder.classes_}")

Category classes: ['ABBR' 'DESC' 'ENTY' 'HUM' 'LOC' 'NUM']
Specific type classes: ['abb' 'animal' 'body' 'city' 'code' 'color' 'count' 'country' 'cremat'
 'currency' 'date' 'def' 'desc' 'dismed' 'dist' 'event' 'exp' 'food' 'gr'
 'ind' 'instru' 'lang' 'letter' 'manner' 'money' 'mount' 'ord' 'other'
 'perc' 'period' 'plant' 'product' 'reason' 'religion' 'speed' 'sport'
 'state' 'substance' 'symbol' 'techmeth' 'temp' 'termeq' 'title' 'veh'
 'volsize' 'weight' 'word']
Combined label classes: ['ABBR_abb' 'ABBR_exp' 'DESC_def' 'DESC_desc' 'DESC_manner' 'DESC_reason'
 'ENTY_animal' 'ENTY_body' 'ENTY_color' 'ENTY_cremat' 'ENTY_currency'
 'ENTY_dismed' 'ENTY_event' 'ENTY_food' 'ENTY_instru' 'ENTY_lang'
 'ENTY_letter' 'ENTY_other' 'ENTY_plant' 'ENTY_product' 'ENTY_religion'
 'ENTY_sport' 'ENTY_substance' 'ENTY_symbol' 'ENTY_techmeth' 'ENTY_termeq'
 'ENTY_veh' 'ENTY_word' 'HUM_desc' 'HUM_gr' 'HUM_ind' 'HUM_title'
 'LOC_city' 'LOC_country' 'LOC_mount' 'LOC_other' 'LOC_state' 'NUM_code'
 'NUM_count

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the Tokenizer and fit it on your training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_set['processed_question'])

# Convert the text data into sequences
X_train_sequences = tokenizer.texts_to_sequences(train_set['processed_question'])
X_test_sequences = tokenizer.texts_to_sequences(test_set['processed_question'])

# Get the maximum sequence length (for padding purposes)
max_sequence_length = max([len(seq) for seq in X_train_sequences])

# Pad the sequences to make them all the same length
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')


In [ ]:
# Extract the labels
y_train = train_set['combined_label_encoded']
y_test = test_set['combined_label_encoded']


In [ ]:
print(X_train_padded.shape)  # Should print (num_train_samples, max_sequence_length)
print(X_test_padded.shape)   # Should print (num_test_samples, max_sequence_length)
print(y_train.shape)         # Should print (num_train_samples,)
print(y_test.shape)          # Should print (num_test_samples,)


(5452, 22)
(500, 22)
(5452,)
(500,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import numpy as np
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=X_train_padded.shape[1]))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(np.unique(y_train)), activation='softmax'))  # Multi-class classification


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    X_train_padded,
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)


Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 20s 87ms/step - accuracy: 0.1803 - loss: 3.3920 - val_accuracy: 0.1220 - val_loss: 3.3321
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 15s 85ms/step - accuracy: 0.2623 - loss: 2.7987 - val_accuracy: 0.4100 - val_loss: 2.5892
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 15s 89ms/step - accuracy: 0.4308 - loss: 2.0480 - val_accuracy: 0.4260 - val_loss: 2.1604
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 21s 95ms/step - accuracy: 0.5177 - loss: 1.6988 - val_accuracy: 0.5020 - val_loss: 2.0167
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 20s 93ms/step - accuracy: 0.6165 - loss: 1.3210 - val_accuracy: 0.5340 - val_loss: 2.0178
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - accuracy: 0.6560 - loss: 1.1904 - val_accuracy: 0.5360 - val_loss: 1.9687
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 20s 94ms/step - accuracy: 0.7424 - loss: 0.9096 - val_accuracy: 0.5900 - val_loss: 1.9781
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 17s 98ms/step - accuracy: 0.7844 - loss: 0.7899 - 

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Accuracy: {test_accuracy}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.6384 - loss: 2.2848
Test Accuracy: 0.6340000033378601


##Zero Shot Classifiers##
Large language models are zero-shot text classifiers

##Bert for Question Classification##
Question and Answer Classification with Deep Contextualized Transformer
Attention is all you need
Attention-Based Transformer-BiGRU for
Question Classification


In [ ]:
import re
import tqdm
import torch
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
#Tokenizer for Bert
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#Dataset
class QuestionDataset(Dataset):
    def __init__(self, questions, labels, max_len=64):
        super(QuestionDataset, self).__init__()
        self.questions = questions
        self.labels = labels
        self.max_len = max_len
    def __getitem__(self, item):
        question = self.questions[item]
        label = self.labels[item]

        # Tokenize question
        encoded_dict = tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids = encoded_dict['input_ids'].squeeze()
        attention_mask = encoded_dict['attention_mask'].squeeze()
        return input_ids, attention_mask, torch.tensor(label)

    def __len__(self):
        return len(self.questions)

In [ ]:
#Modeling
class BertQuestionClassification(nn.Module):
    def __init__(self, n_classes, pretrained_name='bert-base-uncased'):
        super(BertQuestionClassification, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_name)
        self.dropout = nn.Dropout(0.1)
        self.classification = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids, attention_mask)
        out = out.last_hidden_state[:, 0, :]
        out = self.dropout(out)
        out = self.classification(out)

        return out


In [ ]:
#Building methods
def category_to_index(csv_path='/content/Question_Classification_Dataset.csv'):
    df = pd.read_csv(csv_path)
    label = df['Category0']
    category = sorted(set(label.values))
    return {word: index for index, word in enumerate(category)}
def get_data(csv_path='/content/Question_Classification_Dataset.csv'):
    df = pd.read_csv(csv_path)
    questions = list(df['Questions'])
    category = category_to_index(csv_path=csv_path)
    labels = [category[i] for i in df['Category0'].values]
    return questions, labels

In [ ]:
def train_model():
    # Set up device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load data
    questions, labels = get_data()

    # Split data
    X_trainval, X_test, y_trainval, y_test = train_test_split(questions, labels, test_size=0.3, random_state=2021)
    X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.4, random_state=2021)

    # Create datasets and dataloaders
    BATCH_SIZE = 40
    train_dataset = QuestionDataset(X_train, y_train)
    val_dataset = QuestionDataset(X_val, y_val)
    test_dataset = QuestionDataset(X_test, y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    test_dataloader = DataLoader(test_dataset, batch_size=len(X_test))

    # Initialize model and move it to the device
    model = BertQuestionClassification(n_classes=6)
    model = model.to(device)  # Move model to GPU

    # Define optimizer and loss function
    optimizer = Adam(model.parameters(), lr=2e-5)
    loss_fn = CrossEntropyLoss()

    # Training loop
    N_EPOCHS = 20
    MODEL_SAVE_PATH = '/content/model.pth'
    train_losses = []
    val_losses = []
    val_f1scores = []

    for epoch in range(N_EPOCHS):
        print(f"Epoch {epoch + 1}/{N_EPOCHS}")

        # Training
        model.train()
        train_batch_losses = []
        for input_ids, attention_mask, y_train_batch in tqdm.tqdm(train_dataloader):
            # Move data to GPU
            input_ids, attention_mask, y_train_batch = input_ids.to(device), attention_mask.to(device), y_train_batch.to(device)

            y_train_pred = model(input_ids, attention_mask)
            loss = loss_fn(y_train_pred, y_train_batch)
            train_batch_losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        val_batch_losses = []
        val_batch_f1scores = []
        with torch.no_grad():
            for input_ids, attention_mask, y_val_batch in val_dataloader:
                # Move data to GPU
                input_ids, attention_mask, y_val_batch = input_ids.to(device), attention_mask.to(device), y_val_batch.to(device)

                y_val_pred = model(input_ids, attention_mask)
                loss = loss_fn(y_val_pred, y_val_batch)
                val_batch_losses.append(loss.item())

                # Compute F1 score
                predictions = torch.argmax(torch.nn.functional.softmax(y_val_pred, dim=-1), dim=-1)
                val_batch_f1scores.append(
                    f1_score(predictions.cpu(), y_val_batch.cpu(), average='macro')
                )

        # Metrics
        train_losses.append(sum(train_batch_losses) / len(train_batch_losses))
        val_losses.append(sum(val_batch_losses) / len(val_batch_losses))
        val_f1scores.append(sum(val_batch_f1scores) / len(val_batch_f1scores))

        # Save model
        torch.save(model.state_dict(), MODEL_SAVE_PATH)

        print(f"Train Loss: {train_losses[-1]:.4f} | Validation Loss: {val_losses[-1]:.4f} | Validation F1: {val_f1scores[-1]:.4f}")

# Entry point
if __name__ == '__main__':
    train_model()


Using device: cuda
Epoch 1/20


100%|██████████| 58/58 [00:20<00:00,  2.85it/s]


Train Loss: 0.9383 | Validation Loss: 0.3817 | Validation F1: 0.8038
Epoch 2/20


100%|██████████| 58/58 [00:21<00:00,  2.74it/s]


Train Loss: 0.2553 | Validation Loss: 0.2542 | Validation F1: 0.8572
Epoch 3/20


...

Train Loss: 0.0015 | Validation Loss: 0.3473 | Validation F1: 0.9276
Epoch 19/20


100%|██████████| 58/58 [00:21<00:00,  2.72it/s]


Train Loss: 0.0014 | Validation Loss: 0.3623 | Validation F1: 0.9183
Epoch 20/20


100%|██████████| 58/58 [00:21<00:00,  2.72it/s]


Train Loss: 0.0257 | Validation Loss: 0.3423 | Validation F1: 0.9079


In [ ]:
#Prediction


##Word2Vec - ElMo - Embeddings for classification